In [1]:
import pandas as pd
import numpy as np

In [2]:
from pandasql import sqldf

# Flight Data Analysis

# This notebook will guide you through the process of loading, merging, and analyzing the provided airline datasets. 
# We will use `pandas` for data manipulation, `numpy` for numerical operations, and `pandasql` for running SQL queries on our DataFrames.


# 1. Data Loading

bag_level_data = pd.read_csv('Bag+Level+Data.csv')
flight_level_data = pd.read_csv('Flight Level Data.csv')
pnr_remark_level_data = pd.read_csv('PNR Remark Level Data.csv')
pnr_flight_level_data = pd.read_csv('PNR+Flight+Level+Data.csv')
airports_data = pd.read_csv('Airports Data.csv')


In [3]:

# 2. Data Exploration

print("Bag Level Data Info:")
bag_level_data.info()
print("\nFlight Level Data Info:")
flight_level_data.info()
print("\nPNR Remark Level Data Info:")
pnr_remark_level_data.info()
print("\nPNR Flight Level Data Info:")
pnr_flight_level_data.info()
print("\nAirports Data Info:")
airports_data.info()

bag_level_data.head()
flight_level_data.head()
pnr_remark_level_data.head()
pnr_flight_level_data.head()
airports_data.head()


Bag Level Data Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 687245 entries, 0 to 687244
Data columns (total 8 columns):
 #   Column                            Non-Null Count   Dtype 
---  ------                            --------------   ----- 
 0   company_id                        687245 non-null  object
 1   flight_number                     687245 non-null  int64 
 2   scheduled_departure_date_local    687245 non-null  object
 3   scheduled_departure_station_code  687245 non-null  object
 4   scheduled_arrival_station_code    687245 non-null  object
 5   bag_tag_unique_number             687245 non-null  object
 6   bag_tag_issue_date                687245 non-null  object
 7   bag_type                          687245 non-null  object
dtypes: int64(1), object(7)
memory usage: 41.9+ MB

Flight Level Data Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8099 entries, 0 to 8098
Data columns (total 15 columns):
 #   Column                              Non-Null Count 

,airport_iata_code,iso_country_code
0,DAU,PG
1,MDU,PG
2,MXH,PG
3,MIS,PG
4,TIZ,PG


In [4]:

# 3. Data Merging and Preparation

pnr_flight_level_data['scheduled_departure_date_local'] = pd.to_datetime(pnr_flight_level_data['scheduled_departure_date_local'])
flight_level_data['scheduled_departure_date_local'] = pd.to_datetime(flight_level_data['scheduled_departure_date_local'])
bag_level_data['scheduled_departure_date_local'] = pd.to_datetime(bag_level_data['scheduled_departure_date_local'])

merged_data = pd.merge(
    pnr_flight_level_data, 
    flight_level_data, 
    on=['company_id', 'flight_number', 'scheduled_departure_date_local', 'scheduled_departure_station_code', 'scheduled_arrival_station_code'],
    how='left'
)

merged_data = pd.merge(
    merged_data,
    pnr_remark_level_data,
    on=['record_locator', 'flight_number'],
    how='left'
)


In [5]:

# 4. Analysis with Pandasql

pysqldf = lambda q: sqldf(q, globals())

# Example Query 1: Total passengers for each flight

query1 = """
SELECT 
    company_id, 
    flight_number, 
    scheduled_departure_date_local, 
    SUM(total_pax) as total_passengers
FROM 
    pnr_flight_level_data
GROUP BY 
    company_id, 
    flight_number, 
    scheduled_departure_date_local
ORDER BY
    total_passengers DESC
LIMIT 10;
"""

top_flights_by_pax = pysqldf(query1)
print(top_flights_by_pax)

# Example Query 2: Flights with the most special service requests

query2 = """
SELECT 
    flight_number, 
    special_service_request, 
    COUNT(*) as request_count
FROM 
    pnr_remark_level_data
WHERE 
    special_service_request IS NOT NULL
GROUP BY 
    flight_number, 
    special_service_request
ORDER BY
    request_count DESC
LIMIT 10;
"""

top_ssr_flights = pysqldf(query2)
print(top_ssr_flights)

# Example Query 3: Join with Airport Data to get Country Information

query3 = """
SELECT
    t1.company_id,
    t1.flight_number,
    t1.scheduled_departure_station_code,
    t2.iso_country_code as departure_country,
    t1.scheduled_arrival_station_code,
    t3.iso_country_code as arrival_country
FROM
    flight_level_data t1
LEFT JOIN
    airports_data t2 ON t1.scheduled_departure_station_code = t2.airport_iata_code
LEFT JOIN
    airports_data t3 ON t1.scheduled_arrival_station_code = t3.airport_iata_code
LIMIT 10;
"""

flight_country_info = pysqldf(query3)
print(flight_country_info)

  company_id  flight_number scheduled_departure_date_local  total_passengers
0         UA           1599     2025-08-02 00:00:00.000000               530
1         UA            622     2025-08-09 00:00:00.000000               517
2         UA           1599     2025-08-09 00:00:00.000000               510
3         UA            622     2025-08-02 00:00:00.000000               485
4         UA           1387     2025-08-02 00:00:00.000000               481
5         UA           1387     2025-08-10 00:00:00.000000               478
6         UA           1599     2025-08-13 00:00:00.000000               470
7         UA           1599     2025-08-12 00:00:00.000000               466
8         UA           1599     2025-08-03 00:00:00.000000               462
9         UA           1599     2025-08-10 00:00:00.000000               457
   flight_number special_service_request  request_count
0            845      Airport Wheelchair            192
1            973      Airport Wheelchair 